<a href="https://colab.research.google.com/github/pcarbomestre/SAR-VesselDetection-FisheriesMonitoring/blob/main/img_access.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Accessing SENTINEL SAR data

In [ ]:
# Import packages
import os
import ee
import json
import time

In [ ]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=pRe2dvWL2fESiyT7aGgaunLLGlpcSsIL8I1pE1lOckU&tc=vhlgUycdMpHAoQV0-fzTD32o16P_LsdZfZo7wefePaw&cc=4pCjpFlHBYRkzd3QTch7b6zhq6_SCDEXYEi8nD2uSY0

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AbUR2VPNxybq_S1okUR0VS5A-nk7ynCgC4SaYIiQ3QV0amKthI9zRT_hZVM

Successfully saved authorization token.


In [ ]:
# Initialize ee
ee.Initialize()

In [ ]:
# Mount Google Drive (Import files from Google Drive in Colab)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Creating a extraction pipeline

Function for the first image on the date range

In [ ]:
def export_sentinel_image(start_date, end_date, roi_path):

    # Open the GeoJSON file and load its contents
    with open(roi_path) as f:
        geoJSON = json.load(f)

    # Extract the coordinates of the region of interest (ROI) from the GeoJSON file
    coords = geoJSON['features'][0]['geometry']['coordinates']
    aoi = ee.Geometry.Polygon(coords)

    # Get the first Sentinel-1 image for the ROI within the specified date range
    ffa_db = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD')
                           .select(['VV'])
                           .filterBounds(aoi)
                           .filterDate(start_date, end_date)
                           .first()
                           .clip(aoi))

    # Get the timestamp of the image
    date = ee.Date(ffa_db.get('system:time_start')).format("yyyy-MM-dd HH:mm:ss")
    date_dict = date.getInfo()

    # Get the ROI name from the GeoJSON file path
    roi_name = os.path.splitext(os.path.basename(roi_path))[0]

    # Format the output filename with ROI name and timestamp
    date_str = date_dict.replace(" ", "_").replace(":", "")
    file_name = f"{roi_name}_{date_str}"

    # Export the image to Google Drive
    task = ee.batch.Export.image.toDrive(**{
        'image': ffa_db,
        'description': file_name,
        'folder': 'sentinel_images',
        'scale': 10,
    })

    # Start the export task
    task.start()

    # Poll the status of the export task until it is completed
    while task.active():
      print('exporting image',file_name,'(task id: {}).'.format(task.id))
      time.sleep(5)


In [ ]:
roi_path = '/content/drive/Shareddrives/ssdd_pcarbomestre/SSDD_pcarbomestre_2.0/study_case/regions_of_interest/corcovadogulf.geojson'
export_sentinel_image('2019-08-01', '2019-08-30', roi_path)

exporting image corcovadogulf_2019-08-01_235744 (task id: CIVA2IBCULWY4K3KPSS4W2IH).
exporting image corcovadogulf_2019-08-01_235744 (task id: CIVA2IBCULWY4K3KPSS4W2IH).
exporting image corcovadogulf_2019-08-01_235744 (task id: CIVA2IBCULWY4K3KPSS4W2IH).
exporting image corcovadogulf_2019-08-01_235744 (task id: CIVA2IBCULWY4K3KPSS4W2IH).
exporting image corcovadogulf_2019-08-01_235744 (task id: CIVA2IBCULWY4K3KPSS4W2IH).
exporting image corcovadogulf_2019-08-01_235744 (task id: CIVA2IBCULWY4K3KPSS4W2IH).
exporting image corcovadogulf_2019-08-01_235744 (task id: CIVA2IBCULWY4K3KPSS4W2IH).
exporting image corcovadogulf_2019-08-01_235744 (task id: CIVA2IBCULWY4K3KPSS4W2IH).
exporting image corcovadogulf_2019-08-01_235744 (task id: CIVA2IBCULWY4K3KPSS4W2IH).
exporting image corcovadogulf_2019-08-01_235744 (task id: CIVA2IBCULWY4K3KPSS4W2IH).
exporting image corcovadogulf_2019-08-01_235744 (task id: CIVA2IBCULWY4K3KPSS4W2IH).
exporting image corcovadogulf_2019-08-01_235744 (task id: CIVA2IB

# Final image export function

Function for all images in the date range

In [ ]:
def export_sentinel_images(start_date, end_date, roi_path):

    # Open the GeoJSON file and load its contents
    with open(roi_path) as f:
        geoJSON = json.load(f)

    # Extract the coordinates of the region of interest (ROI) from the GeoJSON file
    coords = geoJSON['features'][0]['geometry']['coordinates']
    aoi = ee.Geometry.Polygon(coords)

    # Get a collection of all Sentinel-1 images for the ROI within the specified date range
    s1_coll = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filterBounds(aoi) \
        .filterDate(start_date, end_date) \
        .select(['VV'])

    # Export each image in the collection to Google Drive
    image_list = s1_coll.toList(s1_coll.size())
    num_images = image_list.size().getInfo()

    for i in range(num_images):
        image = ee.Image(image_list.get(i)).clip(aoi)

        # Get the timestamp of the image
        date = ee.Date(image.get('system:time_start')).format("yyyy-MM-dd HH:mm:ss")
        date_dict = date.getInfo()

        # Get the ROI name from the GeoJSON file path
        roi_name = os.path.splitext(os.path.basename(roi_path))[0]

        # Format the output filename with ROI name and timestamp
        date_str = date_dict.replace(" ", "_").replace(":", "")
        file_name = f"{roi_name}_{date_str}"

        # Export the image to Google Drive
        task = ee.batch.Export.image.toDrive(**{
            'image': image,
            'description': file_name,
            'folder': 'sentinel_images',
            'scale': 10,
        })

        # Start the export task
        task.start()

        # Poll the status of the export task until it is completed
        while task.active():
            print('exporting image',file_name,'(task id: {}).'.format(task.id))
            time.sleep(5)


In [ ]:
roi_path = '/content/drive/Shareddrives/ssdd_pcarbomestre/SSDD_pcarbomestre_2.0/study_case/regions_of_interest/corcovadogulf.geojson'

In [ ]:
export_sentinel_images('2022-04-22', '2022-12-31', roi_path)

exporting image corcovadogulf_2022-04-23_235757 (task id: I67RSYU2N4WNMO3ZB57GK7CC).
exporting image corcovadogulf_2022-04-23_235757 (task id: I67RSYU2N4WNMO3ZB57GK7CC).
exporting image corcovadogulf_2022-04-23_235757 (task id: I67RSYU2N4WNMO3ZB57GK7CC).
exporting image corcovadogulf_2022-04-23_235757 (task id: I67RSYU2N4WNMO3ZB57GK7CC).
exporting image corcovadogulf_2022-04-23_235757 (task id: I67RSYU2N4WNMO3ZB57GK7CC).
exporting image corcovadogulf_2022-04-23_235757 (task id: I67RSYU2N4WNMO3ZB57GK7CC).
exporting image corcovadogulf_2022-04-23_235757 (task id: I67RSYU2N4WNMO3ZB57GK7CC).
exporting image corcovadogulf_2022-04-23_235757 (task id: I67RSYU2N4WNMO3ZB57GK7CC).
exporting image corcovadogulf_2022-04-23_235757 (task id: I67RSYU2N4WNMO3ZB57GK7CC).
exporting image corcovadogulf_2022-04-23_235757 (task id: I67RSYU2N4WNMO3ZB57GK7CC).
exporting image corcovadogulf_2022-04-23_235757 (task id: I67RSYU2N4WNMO3ZB57GK7CC).
exporting image corcovadogulf_2022-04-23_235757 (task id: I67RSYU